### The Data files being used
- Manuscript folder now present on the onedrive. 
- I should be moving all of the notebooks to github...

In [ ]:
%qtconsole

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Differential expression

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import rc_context
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
from importlib.machinery import SourceFileLoader 
IM = SourceFileLoader("InterconditionalMethods", "/home/rullman/scRNAmuscleProject/algorithms/scRNAalgoes/src/scMethods.py").load_module() 

In [ ]:
# data = sc.read_h5ad("120521_adata_celltype_prevpost_and_markers")

In [ ]:
projID = "120521"

In [ ]:
data_prepost = IM.diffxpyInterconditionalIterative(
        data, 
        clusterColumn='annotations', 
        projID=projID, 
        use_raw=False, 
        method="wilcoxon"
    )

In [ ]:
batchKeys = ['Endothelial 2_batch', 'Endothelial Cell 0_batch', 'Endothelial Cell 20_batch', 'Mesenchymal cell 11_batch', 'Mesenchymal merged_batch', 'Myoblast 12_batch', 'Myoblast 19_batch', 'Myoblast 3_batch', 'Pericyte 18_batch', 'Pericyte 5_batch', 'Satellite cell_batch', 'T-cells 13_batch', 'T-cells 15_batch']

In [ ]:
IM.add_pct_manual(
    data, 
    clusters = data.obs.annotations.cat.categories,
    clusterColumn="annotations", 
    keys=batchKeys, 
    out=f"{projID}_subclusters_prevpost_rankwilcox.xlsx", 
    save='excel'
) 

> 120521_subclusters_prevpost_rankwilcox.xlsx

In [ ]:
clusters = ["Endothelial 2", "T-cells 13", "Pericyte 5", "Mesenchymal merged", "Endothelial Cell 0", "Bad annotation", "Myoblast 12", "Mesenchymal cell 11", "Myoblast 3", "Satellite cell", "Endothelial Cell 20", "CD14+ Monocyte", "Myoblast 19", "Pericyte 18", "T-cells 15", "CD16+ Monocyte"]
keys = ["Endothelial 2_batch", "T-cells 13_batch", "Pericyte 5_batch", "Mesenchymal merged_batch", "Endothelial Cell 0_batch", "Bad annotation_batch", "Myoblast 12_batch", "Mesenchymal cell 11_batch", "Myoblast 3_batch", "Satellite cell_batch", "Endothelial Cell 20_batch", "CD14+ Monocyte_batch", "Myoblast 19_batch", "Pericyte 18_batch", "T-cells 15_batch", "CD16+ Monocyte_batch"]

In [ ]:
IM.add_pct_manual(
    data, 
    clusters = clusters,
    clusterColumn="annotations", 
    keys=keys, 
    out=f"{projID}_subclusters_prevpost_rankwilcox.xlsx", 
    save='excel'
) 

## Plotting

In [ ]:
plt.rcParams['figure.dpi'] = 600

In [ ]:
palette = ['#393c7b', '#e6b84e', '#8d6d2f', '#657b3a']
lines = [(cl3, ind3), (cl12, ind12)]

In [ ]:
ind12.shape, cl12.shape

In [ ]:
sns.set_style('white')
for ind, s in enumerate(['Myoblast 12', 'Myoblast 3', 'Satellite cell']):
    m  = meta[meta.annotations.str.contains(f'(?i){s}')]
    plt.scatter(m.PC_0, m.PC_1, edgecolor='black', c=palette[ind], linewidths=0.3)

for cl, ind in lines:
    plt.plot(cl['s.X0'].values[ind], cl['s.X1'].values[ind], linewidth=7, color='w')
    plt.plot(cl['s.X0'].values[ind], cl['s.X1'].values[ind], linewidth=5, color='black')

plt.axis('off')
# plt.savefig('210720_MuSC_PCA_plot.png')

In [ ]:
def line_f(to):
    for cl, ind in lines:
        plt.plot(cl['s.X0'].values[ind], cl['s.X1'].values[ind], linewidth=6, color='w')
        plt.plot(cl['s.X0'].values[ind], cl['s.X1'].values[ind], linewidth=5, color='black')
    plt.axis('off')
    plt.savefig(to)
    
line_f('REDO_130321_musc_lines.pdf')
line_f('REDO_130321_musc_lines.png')

In [ ]:
def musc_pts(to):
    sns.set_style('white')
    for ind, s in enumerate(['Myoblast 12', 'Myoblast 3', 'Satellite cell']):
        m  = meta[meta.annotations.str.contains(f'(?i){s}')]
        plt.scatter(m.PC_0, m.PC_1, edgecolor='black', c=palette[ind], linewidths=0.6)

    plt.axis('off')
    plt.savefig(to, dpi=3000)
    
musc_pts('REDO_130321_MuSC_PCA_nolines.png')
musc_pts('REDO_130321_MuSC_PCA_nolines.pdf')

# PCA expression plots

In [ ]:
%qtconsole

### In uppmax

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
# muscvar = sc.read_h5ad('/home/alirassolie/Documents/misc/adata/160720_muscvar.h5ad')
muscvar = sc.read_h5ad("/home/alirassolie/Documents/arbete/mint_mobile/160720_muscvar.h5ad")

In [ ]:
plt.rcParams['figure.dpi'] = 90

In [ ]:
muscvar = muscvar[~muscvar.obs.annotations.str.contains("19")]

In [ ]:
def isolate_PCA_for_genes(out:str, genes:list):
    df_g = pd.DataFrame(index=muscvar.obs.index)
    for gene in genes:
        try:
            gvals = pd.DataFrame({gene:np.stack(muscvar.raw[:, gene].X.toarray(), axis=-1)[0]})
            gvals.index = muscvar.obs.index
            df_g = df_g.join(gvals)
        except KeyError as e:
            print(e)
        #print(df_g)
        #input()

    muscvar.obs = muscvar.obs.join(df_g)

    # adding the pcs to the metadata
    pc = pd.DataFrame(muscvar.obsm['X_pca'])
    pc.columns = [f'PC_{i}' for i in pc.columns]
    pc.index = muscvar.obs.index
    pc = pc.loc[:, 'PC_0':'PC_8']

    muscvar.obs = muscvar.obs.join(pc)
    muscvar.obs.to_csv(out)

In [ ]:
genes = ['TNNC2', 'TNNI2', 'MYLPF', 'TPM1', 'TNNT3', 'ENO3', 'MYL1', 'GAPDH', 'SLN', 'PGAM2', 'CKM', 'TPM2', 'MYOZ1', 'YBX3', 'DES', 'COX7A1', 'ACTA1', 'COX6A2', 'TPT1', 'MYH2', 'TCAP', 'RPLP1', 'ACTB', 'MYBPC2', 'EEF1A2', 'SPARCL1', 'SLC25A4', 'B2M', 'TMSB10', 'EEF1A1', 'MALAT1', 'ATP2A1', 'ITM2B', 'PYGM', 'ACTG1', 'IFITM3', 'CD81', 'LGALS3', 'BIN1', 'HLA-A', 'TMSB4X', 'LGALS1', 'KLHL41', 'PTMA', 'VIM', 'PDLIM3', 'MYL6', 'CHCHD10', 'IGFBP7', 'H3F3B', 'MYBPC1', 'CRYAB', 'UQCRB', 'G0S2', 'TIMP3', 'HLA-E', 'ADSSL1', 'RPL3', 'YBX1', 'ITGB1', 'HLA-C', 'SELENOW', 'S100A11', 'CA3', 'CD63', 'DDX5', 'LDHA', 'PGM1', 'NDUFA4', 'S100A10', 'NACA', 'NDUFB10', 'ATP5F1D', 'MT-ND5', 'GSN', 'RPL37A', 'COX7C', 'MTRNR2L12', 'HLA-B', 'JUND', 'TCF4', 'ANXA2', 'MYL12B', 'KLF2', 'CFL1', 'IDH2', 'CALD1', 'TUBA1B', 'TTN', 'TAGLN2', 'RPS2', 'CAV1', 'TUBA1A', 'BTG1', 'JUN', 'MT-ND4', 'FHL1', 'S100A6', 'SMIM37', 'PKM', 'PPIA', 'PDLIM7', 'CMYA5', 'MT-ND3', 'S100A1', 'MT-CYB', 'SON', 'NEAT1', 'MT-CO1', 'RAD23A', 'CDKN1C', 'H3F3A', 'CD59', 'TMBIM6', 'DYNLL1', 'ANXA5', 'CLIC1', 'FHL3', 'SPARC', 'TPI1', 'SPTBN1', 'LAPTM4A', 'TCEA3', 'MTRNR2L8', 'NEB', 'RPL3L', 'GNAI2', 'ITM2A', 'EEF1B2', 'ALDOA', 'CST3', 'STAC3', 'PTMS', 'RPS16', 'SEPT7', 'IER2', 'CLEC14A', 'MT-ND2', 'ACTN2', 'SEC62', 'APP', 'MT-ATP6', 'UQCR11', 'CAVIN1', 'GNG11', 'NRAP', 'AHNAK', 'PPIB', 'HSPG2', 'ANK1', 'ADIRF', 'IFITM2', 'JUNB', 'HSP90B1', 'CDC42', 'COX5A', 'APOBEC2', 'MGLL', 'ZFP36L2', 'YWHAB', 'CD99', 'RHOC', 'PPP1R1A', 'MT-CO2', 'UQCR10', 'ZFP36L1', 'CALR', 'HMGB1', 'DNAJA1', 'PDIA3PDLIM3', 'PSAP', 'RPL38', 'TOMM7', 'DUSP1', 'FOSB', 'CD151', 'DSTN', 'IFI27', 'TACC1', 'NDUFV2', 'TXNIP', 'COX6A1', 'COL4A1', 'MACROD1', 'MT-ND4L', 'SELENOP', 'BST2', 'MT-CO3', 'EMP2', 'MARCKS', 'SRSF3', 'PPP1R27', 'FXYD1', 'HSP90AA1', 'PPDPFL', 'UQCRFS1', 'MYL9', 'DPYSL2', 'RAMP2', 'CYR61', 'WSB1', 'STOM', 'KLF6', 'MT2A', 'MT-ND1', 'UBC', 'PDLIM5', 'FUS', 'DAD1', 'ZBTB20', 'A2M', 'TAGLN', 'RPL13A', 'TRDN', 'MIR133A1HG', 'PDE4DIP', 'MYOT', 'NEXN', 'ATP2A2', 'RBM24', 'MYH7', 'SVIL', 'TNNT1', 'TNNI1', 'EMC10', 'MB', 'LDB3', 'ATP1A2', 'XIRP2', 'LMOD3', 'MYL2', 'TPM3', 'ATP1B1', 'LMOD2', 'PPP1R3A', 'TNNC1', 'AGL', 'TMEM38A', 'FBXO32', 'COQ10A', 'TRIM54', 'POLR2J3-1', 'DWORF', 'CKMT2', 'CSRP3', 'MYL3', 'HSPB7', 'SMPX', 'PEBP4', 'CYB5R1', 'MDH1', 'CASQ1', 'FILIP1', 'AC020909.2', 'MYH7B', 'DMD', 'FABP3', 'TXLNB', 'GAMT', 'CACNG1', 'SCN1B', 'RRAD', 'LRRC39', 'CORO6', 'HOOK2', 'LINC01405', 'PLN', 'RETREG1', 'AMPD1', 'COQ8A', 'LRRC2', 'GOT1', 'FITM1', 'NPHP1', 'MIR1-1HG', 'MYF6', 'RTN2', 'ASB5', 'CFL2', 'PKIA', 'ATP5MC1', 'PDK4', 'N4BP2L2', 'LMCD1', 'SYNPO2', 'MEF2C', 'SIX1', 'MYOM1', 'OBSCN', 'MUC20-OT1', 'FLNC', 'HSPB6', 'CAP2', 'SGCA', 'VDAC1', 'TUBA4A', 'ANKRD9', 'RAMP1', 'MBNL1', 'CAVIN4', 'ZNF106', 'DUSP13', 'SH3BGR', 'IDI2', 'TRIM7', 'PLIN5', 'NDUFV3', 'ANKRD2', 'ANKRD23', 'CEBPB', 'VDAC3', 'MYPN', 'AL451062.1', 'DUSP26', 'ASPH', 'CUTC', 'ASB2', 'ART3', 'NIPSNAP2', 'MYL6B', 'HHATL']
genes = list(Counter(genes).keys())
isolate_PCA_for_genes("./musc/REDO_130321_muscvar_metadata.csv", genes)

In [ ]:
new_genes = ['CXCL8', 'MYOD1',  'HSP90AA1','HSP90AB1', 'HDAC4', "MYH8", "LMNA", "NCL", "NFE2L2", "NNMT", "SOD2","NFKBIA"]

isolate_PCA_for_genes("./musc/REDO_300321_muscvar_metadata.csv", new_genes)

### Visualizing genes pre and post

In [ ]:
muscvar_pre = muscvar[muscvar.obs.batch == "0"]
muscvar_post = muscvar[muscvar.obs.batch == "1"]

#### PCA-loadings

In [ ]:
sc.tl.pca(muscvar)

In [ ]:
sc.pl.pca_loadings(muscvar)

In [ ]:
PC_loadings = pd.DataFrame(muscvar.varm['PCs'])
PC_loadings.index = muscvar.var.index

In [ ]:
tmp = PC_loadings.sort_values(0, ascending=False)[:100]
tmp.loc[:, 1] = tmp.loc[:, 1] * -1
fig, axs = plt.subplots(ncols=1)
axs.scatter(tmp[0], tmp[1], s=8)

for i, text in enumerate(tmp.index):
    axs.text(tmp[0][i], tmp[1][i], s=text, size=6)
    
#axs.set_xlim()

#### PCA-plot with expression level

In [ ]:
import matplotlib.cm as cm
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px


In [ ]:
%matplotlib notebook

In [ ]:
muscvar_pre = muscvar[muscvar.obs.batch == "0"]
muscvar_post = muscvar[muscvar.obs.batch == "1"]

In [ ]:
muscDF = pd.DataFrame(muscvar.obsm["X_pca"])
muscDF_pre = pd.DataFrame(muscvar_pre.obsm["X_pca"])
muscDF_post = pd.DataFrame(muscvar_post.obsm["X_pca"])

In [ ]:
muscvar[:, "MYF5"].X.flatten()

In [ ]:
def map_PCA_to_RGB(data:"AnnData", gene:str):
    """
    """
    try: gene_counts = data[:, gene].X.flatten()
    except: gene_counts = data[:, gene].X.toarray().flatten()
    
    minima = min(gene_counts)
    maxima = max(gene_counts)

    norm = mpl.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.hot)
    
    return [mapper.to_rgba(v) for v in gene_counts]


In [ ]:
def add_color_column_by_gene(df, anndata, gene):
    try: df[gene] = anndata[:, gene].X.flatten()
    except: df[gene] = anndata[:, gene].X.toarray().flatten()
    return df

In [ ]:
def PCA_scatter_3D(df, anndata, gene):
    df_c_added = add_color_column_by_gene(df=df, anndata=anndata, gene=gene)
    fig = px.scatter_3d(df_c_added, x=0, y=1, z=2, color=gene)
    fig.update_traces(marker={'size': 3})

    fig.show()

In [ ]:
def PCA_scatter_3D_batch(df, anndata, gene, **kwargs):
    df['batch'] = anndata.obs.batch.values
    fig = px.scatter_3d(df, x=0, y=1, z=2, color="batch", **kwargs)
    fig.update_traces(marker={'size': 3})

    fig.show()

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(15,5))
axs[0].scatter(muscDF_pre.loc[:, 3]*-1, muscDF_pre.loc[:, 1], color=map_PCA_to_RGB(muscvar_pre, "TNNI1"), s=2)
axs[1].scatter(muscDF_post.loc[:, 3]*-1, muscDF_post.loc[:, 1], color=map_PCA_to_RGB(muscvar_post, "TNNI2"), s=2)

In [ ]:
sc.tl.pca(muscvar[:, ])

In [ ]:
PCA_scatter_3D_batch(muscDF, muscvar, "MYOD1", palette=["black", "gray"])

In [ ]:
PCA_scatter_3D(muscDF_post, muscvar_post, "MYOG")

In [ ]:
PCA_scatter_3D(muscDF, muscvar, "MYF6")

In [ ]:
plt.rcParams["figure.dpi"] = 300
sc.pl.pca(muscvar, color=["batch"], palette=["lightgray", "lightblue"], edgecolor="black", linewidth=0.05, s=30)

In [ ]:
muscvar[:, "MRF4"]

### Run locally to generate the figures after importing the PCA metadata with the coordinates

In [ ]:
def musc_expr(genes, anndata, to=None, out=None, figsize=(10, 20)):
    """Plot the expression of the myoblasts along the PCA axes
    Args:
        genes (dict)
        meta (pandas df):   contains the metadata with the 
                            PCA coordinates
                            
    Kwargs: 
        to (str)
        out (str)
    """
    
    #plt.hist(meta.TNNI2[meta.TNNI2 > 15], bins=100)
    plt.rcParams['axes.facecolor'] = 'black'
    gene_indices = anndata.var_names.values
    count_matrix = anndata.X.toarray()
    baseline_pca = pd.DataFrame(anndata.obsm['X_pca'])
    for i, gene in enumerate(genes.keys()):

        fig, axs = plt.subplots(nrows=1, figsize=figsize)
        filt_ind = count_matrix[:, np.where(gene_indices == gene)[0][0]] > genes[gene][0]

        meta = pd.DataFrame(anndata[filt_ind].obsm['X_pca']) 
        axs.grid(False)
        axs.scatter(baseline_pca[0], baseline_pca[1], c='gray', s=10, alpha=0.5, linewidth=0)
        axs.scatter(meta[0], meta[1], c=expr_colors[i % len(expr_colors)], s= 10,  **expr_params)
        axs.set_title(gene)

    
    plt.show()
        

    
# musc_expr('REDO_130321_MuSC_PCA_markers.pdf')

#### 130321 genes of interest

In [ ]:

expr_params = {
    "edgecolor": "black",
    "linewidth": 0.3,
    "alpha":0.5,
}

genes = {
    "TNNI2": [3.6], 
    "TNNI1": [2.4],
    "PAX7": [0]
    #'TNNI2':[0], 'MYLPF':[0], 'TPM1':[0], 'TNNT3':[0], 'ENO3':[0], 'MYL1':[0], 'GAPDH':[0], 'SLN':[0], 'PGAM2':[0], 'CKM':[0], 'TPM2':[0], 'MYOZ1':[0], 'YBX3':[0], 'DES':[0], 'COX7A1':[0], 'ACTA1':[0], 'COX6A2':[0], 'TPT1':[0], 'MYH2':[0], 'TCAP':[0], 'RPLP1':[0], 'ACTB':[0], 'MYBPC2':[0], 'EEF1A2':[0], 'SPARCL1':[0], 'SLC25A4':[0], 'B2M':[0], 'TMSB10':[0], 'EEF1A1':[0], 'MALAT1':[0], 'ATP2A1':[0], 'ITM2B':[0], 'PYGM':[0], 'ACTG1':[0], 'IFITM3':[0], 'CD81':[0], 'LGALS3':[0], 'BIN1':[0], 'HLA-A':[0], 'TMSB4X':[0], 'LGALS1':[0], 'KLHL41':[0], 'PTMA':[0], 'VIM':[0], 'PDLIM3':[0], 'MYL6':[0], 'CHCHD10':[0], 'IGFBP7':[0], 'H3F3B':[0], 'MYBPC1':[0], 'CRYAB':[0], 'UQCRB':[0], 'G0S2':[0], 'TIMP3':[0], 'HLA-E':[0], 'ADSSL1':[0], 'RPL3':[0], 'YBX1':[0], 'ITGB1':[0], 'HLA-C':[0], 'SELENOW':[0], 'S100A11':[0], 'CA3':[0], 'CD63':[0], 'DDX5':[0], 'LDHA':[0], 'PGM1':[0], 'NDUFA4':[0], 'S100A10':[0], 'NACA':[0], 'NDUFB10':[0], 'ATP5F1D':[0], 'MT-ND5':[0], 'GSN':[0], 'RPL37A':[0], 'COX7C':[0], 'MTRNR2L12':[0], 'HLA-B':[0], 'JUND':[0], 'TCF4':[0], 'ANXA2':[0], 'MYL12B':[0], 'KLF2':[0], 'CFL1':[0], 'IDH2':[0], 'CALD1':[0], 'TUBA1B':[0], 'TTN':[0], 'TAGLN2':[0], 'RPS2':[0], 'CAV1':[0], 'TUBA1A':[0], 'BTG1':[0], 'JUN':[0], 'MT-ND4':[0], 'FHL1':[0], 'S100A6':[0], 'SMIM37':[0], 'PKM':[0], 'PPIA':[0], 'PDLIM7':[0], 'CMYA5':[0], 'MT-ND3':[0], 'S100A1':[0], 'MT-CYB':[0], 'SON':[0], 'NEAT1':[0], 'MT-CO1':[0], 'RAD23A':[0], 'CDKN1C':[0], 'H3F3A':[0], 'CD59':[0], 'TMBIM6':[0], 'DYNLL1':[0], 'ANXA5':[0], 'CLIC1':[0], 'FHL3':[0], 'SPARC':[0], 'TPI1':[0], 'SPTBN1':[0], 'LAPTM4A':[0], 'TCEA3':[0], 'MTRNR2L8':[0], 'NEB':[0], 'RPL3L':[0], 'GNAI2':[0], 'ITM2A':[0], 'EEF1B2':[0], 'ALDOA':[0], 'CST3':[0], 'STAC3':[0], 'PTMS':[0], 'RPS16':[0], 'SEPT7':[0], 'IER2':[0], 'CLEC14A':[0], 'MT-ND2':[0], 'ACTN2':[0], 'SEC62':[0], 'APP':[0], 'MT-ATP6':[0], 'UQCR11':[0], 'CAVIN1':[0], 'GNG11':[0], 'NRAP':[0], 'AHNAK':[0], 'PPIB':[0], 'HSPG2':[0], 'ANK1':[0], 'ADIRF':[0], 'IFITM2':[0], 'JUNB':[0], 'HSP90B1':[0], 'CDC42':[0], 'COX5A':[0], 'APOBEC2':[0], 'MGLL':[0], 'ZFP36L2':[0], 'YWHAB':[0], 'CD99':[0], 'RHOC':[0], 'PPP1R1A':[0], 'MT-CO2':[0], 'UQCR10':[0], 'ZFP36L1':[0], 'CALR':[0], 'HMGB1':[0], 'DNAJA1':[0], 'PDIA3PDLIM3':[0], 'PSAP':[0], 'RPL38':[0], 'TOMM7':[0], 'DUSP1':[0], 'FOSB':[0], 'CD151':[0], 'DSTN':[0], 'IFI27':[0], 'TACC1':[0], 'NDUFV2':[0], 'TXNIP':[0], 'COX6A1':[0], 'COL4A1':[0], 'MACROD1':[0], 'MT-ND4L':[0], 'SELENOP':[0], 'BST2':[0], 'MT-CO3':[0], 'EMP2':[0], 'MARCKS':[0], 'SRSF3':[0], 'PPP1R27':[0], 'FXYD1':[0], 'HSP90AA1':[0], 'PPDPFL':[0], 'UQCRFS1':[0], 'MYL9':[0], 'DPYSL2':[0], 'RAMP2':[0], 'CYR61':[0], 'WSB1':[0], 'STOM':[0], 'KLF6':[0], 'MT2A':[0], 'MT-ND1':[0], 'UBC':[0], 'PDLIM5':[0], 'FUS':[0], 'DAD1':[0], 'ZBTB20':[0], 'A2M':[0], 'TAGLN':[0], 'RPL13A':[0], 'TRDN':[0], 'MIR133A1HG':[0], 'PDE4DIP':[0], 'MYOT':[0], 'NEXN':[0], 'ATP2A2':[0], 'RBM24':[0], 'MYH7':[0], 'SVIL':[0], 'TNNT1':[0], 'TNNI1':[0], 'EMC10':[0], 'MB':[0], 'LDB3':[0], 'ATP1A2':[0], 'XIRP2':[0], 'LMOD3':[0], 'MYL2':[0], 'TPM3':[0], 'ATP1B1':[0], 'LMOD2':[0], 'PPP1R3A':[0], 'TNNC1':[0], 'AGL':[0], 'TMEM38A':[0], 'FBXO32':[0], 'COQ10A':[0], 'TRIM54':[0], 'POLR2J3-1':[0], 'DWORF':[0], 'CKMT2':[0], 'CSRP3':[0], 'MYL3':[0], 'HSPB7':[0], 'SMPX':[0], 'PEBP4':[0], 'CYB5R1':[0], 'MDH1':[0], 'CASQ1':[0], 'FILIP1':[0], 'AC020909.2':[0], 'MYH7B':[0], 'DMD':[0], 'FABP3':[0], 'TXLNB':[0], 'GAMT':[0], 'CACNG1':[0], 'SCN1B':[0], 'RRAD':[0], 'LRRC39':[0], 'CORO6':[0], 'HOOK2':[0], 'LINC01405':[0], 'PLN':[0], 'RETREG1':[0], 'AMPD1':[0], 'COQ8A':[0], 'LRRC2':[0], 'GOT1':[0], 'FITM1':[0], 'NPHP1':[0], 'MIR1-1HG':[0], 'MYF6':[0], 'RTN2':[0], 'ASB5':[0], 'CFL2':[0], 'PKIA':[0], 'ATP5MC1':[0], 'PDK4':[0], 'N4BP2L2':[0], 'LMCD1':[0], 'SYNPO2':[0], 'MEF2C':[0], 'SIX1':[0], 'MYOM1':[0], 'OBSCN':[0], 'MUC20-OT1':[0], 'FLNC':[0], 'HSPB6':[0], 'CAP2':[0], 'SGCA':[0], 'VDAC1':[0], 'TUBA4A':[0], 'ANKRD9':[0], 'RAMP1':[0], 'MBNL1':[0], 'CAVIN4':[0], 'ZNF106':[0], 'DUSP13':[0], 'SH3BGR':[0], 'IDI2':[0], 'TRIM7':[0], 'PLIN5':[0], 'NDUFV3':[0], 'ANKRD2':[0], 'ANKRD23':[0], 'CEBPB':[0], 'VDAC3':[0], 'MYPN':[0], 'AL451062.1':[0], 'DUSP26':[0], 'ASPH':[0], 'CUTC':[0], 'ASB2':[0], 'ART3':[0], 'NIPSNAP2':[0], 'MYL6B':[0], 'HHATL':[0]
    }

expr_colors = ["#67b1ab", "yellow", "#8d4b91"]


#### 300321 genes of interest

In [ ]:
new_genes = ['CXCL8', 'MYOD1',  'HSP90AA1','HSP90AB1', 'HDAC4', "MYH8", "LMNA", "NCL", "NFE2L2", "NNMT", "SOD2","NFKBIA"]
new_genes_params = {i:[0] for i in new_genes}
expr_colors = ["#67b1ab", "yellow", "#8d4b91"]

In [ ]:
musc_expr(genes=genes, anndata=muscvar, figsize=(6,3))
#count_matrix = muscvar.raw.X.toarray()
#count_matrix[:, np.where(muscvar.raw.var_names.values == "MYF6")[0][0]] > 0


In [ ]:
musc_expr('./figout/130321/expr_musc_130321/REDO_130321_MuSC_PCA_markers', out="png")
musc_expr('./figout/130321/expr_musc_130321/REDO_130321_MuSC_PCA_markers', out="pdf")

# Expression boxplot
Boxplots for the expression of select genes pre and post-exericse, to elaborate on the demultiplexing of convoluted data

## Figure 3 barplots logfoldchanges

### Doing violins instead

In [ ]:
data = sc.read_h5ad("/home/alirassolie/Documents/arbete/mint_mobile/120521_adata_celltype_prevpost_and_markers")
data_filtered = data[data.obs.annotations.str.contains("(?i)sat|myo")]
muscvar = sc.read_h5ad('/home/alirassolie/Documents/arbete/mint_mobile/160720_muscvar.h5ad')
muscvar = muscvar[~muscvar.obs.annotations.str.contains("19")]

In [ ]:
import seaborn as sns
import random

In [ ]:
def test_log1p(x): 
    return np.subtract(np.power(np.e, x), 1)

def test_log1p_apply(array):
    return np.apply_along_axis(test_log1p, 0, array)

def test_log1p_countsnormalize(array):
    return np.apply_along_axis(test_log1p, 0, array).sum()

def test_random_log1p_countsnormalize(adata:'AnnData, AnnData.X == sparse'):
    '''Will take an adata object, where the AnnData.X object
    neeeds to be a sparse matrix'''
    ind = random.randint(0, adata.shape[0])
    t = adata.X[ind, :].toarray()
    result = test_log1p_countsnormalize(t)
    print(f'Testing: UMI index == {ind}')
    print(f'Outcome: {result}')
    return result

def eDiffData(
		adata:'AnnData', 
		key:str = 'rank_genes_groups'
		#cluster:str = 'Muscle cl5',  
		)-> pd.DataFrame:
	#
	# eDiffData
	# Extracts the diffexp dataset and returns a dict
	# which can be converted into a pd.DataFrame. 
	#
	print(f'[eDiffData] Running for: {key}')
	cat = ['scores', 'names', 'logfoldchanges', 'pvals', 'pvals_adj']
	data =[]
	for c in cat:
		print(f'[eDiffData] iterating: {c}')
		tmp = pd.DataFrame(adata.uns[key][c])
		tmp.columns = pd.Index([(tmp.columns[0], c)])
		data.append(tmp)
	# Instead of concating on rows, we do along
	# columns, knowing each column is unique
	# so no issues will be had. 
	data = pd.concat(data, axis=1)
	return data

In [ ]:
for i in range(10):
    print(test_random_log1p_countsnormalize(data_filtered))

In [ ]:
data_filtered.obs.annotations

In [ ]:
sc.tl.rank_genes_groups(data_filtered, 'annotations', groups=['Satellite cell'], reference='Myoblast 12', 
                        method='wilcoxon', key_added="sat_12", use_raw=False)

In [ ]:
eDiffData(data_filtered, "sat_12")

In [ ]:
rn_list = multipanel_violin(data_filtered, genes=["APOE", "RPL3", "EEF1A1"], ncols=3, nrows=3, tick_right=True, figsize=(4,4), toarray=True)

In [ ]:
def gene_distrib(d, genes, s_str, cluster_column):
    """Returns dataframe with data molten for each gene and a given
    column
    
    Args:
        d: AnnData scanpy object
        
        genes: list object of strings corresponding to gene symbols
        
        s_str: string to filter d
    
    """
    tm = d[d.obs[cluster_column].str.contains(s_str)]
    g_vals = pd.DataFrame(tm[:, genes].X)
    g_vals.columns = genes
    g_vals['batch'] = tm.obs.batch.values
    g_vals[cluster_column] = s_str
    g_vals_melt = pd.melt(g_vals, ['batch', cluster_column])
    return g_vals_melt

In [ ]:
def anndata_for_violin(anndata, gene="MYF5", toarray:bool=False):
    geneAnndata = anndata[:, gene]
    if not toarray: geneDF = pd.DataFrame({"normcount": geneAnndata.X.flatten()})
    elif toarray: geneDF = pd.DataFrame({"normcount": geneAnndata.X.toarray().flatten()})
    geneDF["batch"] = anndata.obs.batch.values
    geneDF["label"] = anndata.obs.annotations.values
    geneDF["celltag"] = anndata.obs.index.values
    geneDF["gene"] = gene
    return geneDF

In [ ]:
def multipanel_violin(
        anndata, 
        genes, 
        ncols=2, 
        nrows=2, 
        tick_right=False, 
        toarray=False,
        **kwargs
):
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, **kwargs)
    axs = axs.flatten()
    geneDFs = []
    for i,g in enumerate(genes):
        geneDF = anndata_for_violin(anndata, g, toarray=toarray)

        sns.violinplot(
            data=geneDF,
            x="label",
            y="normcount",
            #hue="batch",
            #split=True,
            ax=axs[i],
            legend=None,
            inner="box",
            palette=["#846934", "#374684", "#dfb359"],
            scale="width",
            cut=0,
            order=['Satellite cell', 'Myoblast 12', 'Myoblast 3']
        )

        axs[i].set_ylim(ymin=0)
        axs[i].legend([],[], frameon=False)
        axs[i].set_xticks([])
        axs[i].set_xlabel(g)
        axs[i].set_ylabel("")
        if tick_right: axs[i].yaxis.tick_right()
        plt.tight_layout()
        geneDFs.append(geneDF)
    
    return geneDFs


In [ ]:
def multipanel_violin_prepost(
        anndata, 
        genes, 
        ncols=2, 
        nrows=2, 
        toarray=False, 
        palette=["#846934", "#374684", "#dfb359", "#ffffff"],
        **kwargs
):
    fig, axs = plt.subplots(ncols=ncols, nrows=nrows, **kwargs)
    if nrows>1: axs = axs.flatten()
    elif nrows==1 and ncols==1: axs = [axs]
    geneDFs = []
    for i,g in enumerate(genes):
        geneDF = anndata_for_violin(anndata, g, toarray=toarray)
        
        sns.violinplot(
            data=geneDF,
            x="label",
            y="normcount",
            hue="batch",
            split=True,
            ax=axs[i],
            legend="off",
            inner=None,
            palette=palette,
            scale="width",
            cut=0,
            order=['Satellite cell', 'Myoblast 12', 'Myoblast 3']
        )
        for k, violin in enumerate(axs[i].findobj(mpl.collections.PolyCollection)):
            if k % 2:
                violin.set_hatch("////")

        #axs[i].legend_.findobj(mpl.patches.Rectangle)[1].set_hatch("///")
        axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=30)

        axs[i].set_ylim(ymin=0)
        
        axs[i].set_xticks([])
        axs[i].set_xlabel(g)
        axs[i].set_ylabel("")
        plt.tight_layout()
        geneDFs.append(geneDF)
        #axs[i].get_legend().remove()
        axs[i].legend([],[], frameon=False)
    
    return geneDFs


In [ ]:
def multigrid_violin_prepost(
        anndata, 
        genes, 
        axs,
        ncols=2, 
        nrows=2, 
        toarray=False, 
        palette=["#846934", "#374684", "#dfb359", "#ffffff"],
        **kwargs
):
    """Visualize pre vs post for myoblasts.
    """
    # fig, axs = plt.subplots(ncols=ncols, nrows=nrows, **kwargs)
    #if nrows>1: axs = axs.flatten()
    #elif nrows==1 and ncols==1: axs = [axs]
    geneDFs = []
    for i,g in enumerate(genes):
        geneDF = anndata_for_violin(anndata, g, toarray=toarray)
        
        sns.violinplot(
            data=geneDF,
            x="label",
            y="normcount",
            hue="batch",
            split=True,
            ax=axs[i],
            legend="off",
            inner=None,
            palette=palette,
            scale="width",
            cut=0,
            # order=['Satellite cell', 'Myoblast 12', 'Myoblast 3']
        )
        for k, violin in enumerate(axs[i].findobj(mpl.collections.PolyCollection)):
            if k % 2:
                violin.set_hatch("////")

        axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=30)

        #axs[i].set_ylim(ymin=0)
        
        axs[i].set_xticks([])
        axs[i].set_xlabel(g)
        axs[i].set_ylabel("")
        plt.tight_layout()
        geneDFs.append(geneDF)
        axs[i].legend([],[], frameon=False)
    
    return geneDFs


#### Nine genes

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [ ]:
top9 = PC_loadings.sort_values(0, ascending=False)[:9]
rn_list = multipanel_violin(muscvar, genes=top9.index, ncols=3, nrows=3, tick_right=True, figsize=(4,4))
# plt.savefig("/home/alirassolie/Documents/misc/fig3_pca_1_vio_9genes.pdf")

In [ ]:
top9 = PC_loadings.sort_values(1, ascending=False)[:9]
rn = multipanel_violin(muscvar, genes=top9.index, ncols=3, nrows=3, figsize=(4,4))
#plt.savefig("/home/alirassolie/Documents/misc/fig3_pca_2_vio.pdf")

##### Selected markers for violins

Satelliter

In [ ]:
# top9 = PC_loadings.sort_values(2, ascending=False)[:9]
rn = multipanel_violin(data_filtered, genes=["PAX7", "APOE", "IGFBP5", "MYF5", "NCAM1", "CHRNA1"], ncols=3, nrows=3, figsize=(4,4), toarray=True, tick_right=True)
plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Figure_4_marker_violins_satellite_specific.pdf")

Myoblaster

In [ ]:
# top9 = PC_loadings.sort_values(2, ascending=False)[:9]
rn = multipanel_violin(data_filtered, genes=["ENO3", "TNNI2", "TNNT3", "MYL2", "TNNT1", "TNNC1"], ncols=3, nrows=3, figsize=(4,4), toarray=True)
plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Figure_4_marker_violins_myoblast_specific.pdf")

In [ ]:
suppGenes = ["ENO3", "TNNI2", "TNNT3", "MYL2", "TNNT1", "TNNC1", "PAX7", "APOE", "IGFBP5", "MYF5", "NCAM1", "CHRNA1"]
dfList = []
for g in suppGenes:
    dfList.append(anndata_for_violin(data_filtered, gene=g, toarray=True))
    
suppDf = pd.concat(dfList)

In [ ]:
writer = pd.ExcelWriter("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Supplement_10_Figure_4_boxplot_inner_290922.xlsx", engine="xlsxwriter")
suppDf.to_excel(writer, sheet_name="Figure_4_violin_data")
writer.save()

#### Four genes

In [ ]:
top4 = PC_loadings.sort_values(0, ascending=False)[:4]
rn_list = multipanel_violin(muscvar, genes=top4.index, ncols=2, nrows=2, figsize=(2,2))
#plt.savefig("/home/alirassolie/Documents/misc/fig3_pca_1_vio.pdf")

#### Eight genes

In [ ]:
top8 = PC_loadings.sort_values(0, ascending=False)[:8]
rn_list = multipanel_violin(muscvar, genes=top8.index, ncols=4, nrows=2, tick_right=True, figsize=(4,2))
plt.savefig("/home/alirassolie/Documents/misc/fig3_pca_1_vio_8genes.pdf")

In [ ]:
top8 = PC_loadings.sort_values(1, ascending=False)[:8]
rn = multipanel_violin(muscvar, genes=top8.index, ncols=4, nrows=2, figsize=(4,2))
plt.savefig("/home/alirassolie/Documents/misc/fig3_pca_2_vio_8genes.pdf")

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [ ]:
tmp = muscvar[muscvar.obs.annotations.str.contains("Sat")]
sat = pd.DataFrame(tmp[:, ""].X.flatten())
sat['batch'] = tmp.obs.batch.values
sat['celltype'] = "Satellite Cells"

fig, axs = plt.subplots(ncols=1, figsize=(2,4))
sns.violinplot(data=sat, y=0, x="celltype", hue='batch', split=True, ax=axs, inner=None)
axs.set_ylim(ymin=0)
axs.legend([],[], frameon=False)

### Pre v post

In [ ]:
sat = pd.read_csv('/home/alirassolie/Documents/misc/sat_prevpost.tsv', 
                  names=["gene", "score", "fc", "pval", "pval_adj"],
                  delimiter='\t')
slow = pd.read_csv('/home/alirassolie/Documents/misc/slow_prevpost.tsv', 
                  names=["gene", "score", "fc", "pval", "pval_adj"],
                  delimiter='\t')
fast = pd.read_csv('/home/alirassolie/Documents/misc/fast_prevpost.tsv', 
                  names=["gene", "score", "fc", "pval", "pval_adj"],
                  delimiter='\t')
slow_up = slow[(slow.fc > 0)
              & (slow.pval_adj <= 0.05)
              ]

sat_down = sat[(sat.fc < 0)
                & (sat.pval_adj <= 0.05)]

slow_down = slow[(slow.fc < 0)
              & (slow.pval_adj <= 0.05)
              ]

sat_up = sat[(sat.fc > 0)
                & (sat.pval_adj <= 0.05)]




sat_down[sat_down.gene.isin(slow_up.gene)]

sat_up[sat_up.gene.isin(slow_down.gene)]

In [ ]:
genes=["NNMT", "NEAT1", "LMNA","MYOZ1",  "NR4A1", "PDK4", "MT1X", "SPARCL1", "CKM","ACTA1",]

In [ ]:
ncols = 19
nrows = 1
figsize = (20,1.3)

In [ ]:
fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
_ = multigrid_violin_prepost(data_filtered[data_filtered.obs.annotations.str.contains("12")], genes=genes, axs=axs, toarray=True)
#plt.savefig("/home/alirassolie/Documents/misc/fast_prepost_vio.pdf")
plt.show()

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
_ = multigrid_violin_prepost(data_filtered[data_filtered.obs.annotations.str.contains("Sat")], genes=genes, axs=axs, toarray=True)
#plt.savefig("/home/alirassolie/Documents/misc/satellites_prepost_vio.pdf")
plt.show()

fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
_ = multigrid_violin_prepost(data_filtered[data_filtered.obs.annotations.str.contains("3")], genes=genes, axs=axs, toarray=True)
#plt.savefig("/home/alirassolie/Documents/misc/slow_prepost_vio.pdf")
plt.show()

In [ ]:
genes = ["ACTA1", "ACTN2", "MYL2", "MYOZ1", "TCAP", "TTN", "MYH7", "XIRP2", "DES", "TPM2", "TNNT1", "MYBPC1", "NEB", "TNNC2", "HSP90AA1", "SCN1B", "NEXN", "BIN1", "TTN"]
_ = multipanel_violin_prepost(data_filtered, genes, nrows=5, ncols=4, toarray=True, figsize = (10,10))


# Selecting cells based on marker positivity and negativity

In [ ]:
PAX7_neg_post = muscvar[
    (muscvar.raw[:, "PAX7"].X.toarray().flatten() == 0)
    & (muscvar.obs.batch == "1")]

In [ ]:
PAX7_neg_pre = muscvar[
    (muscvar.raw[:, "PAX7"].X.toarray().flatten() == 0)
    & (muscvar.obs.batch == "0")]

In [ ]:
sc.pl.pca(PAX7_neg)

In [ ]:
MYOD1_pos_pre = muscvar[
    (muscvar.raw[:, "MYOD1"].X.toarray().flatten() > 0)
    & (muscvar.obs.batch == "0")]

In [ ]:
MYOD1_pos_post = muscvar[
    (muscvar.raw[:, "MYOD1"].X.toarray().flatten() > 0)
    & (muscvar.obs.batch == "1")]

In [ ]:
MYOD1_pos = muscvar[
    (muscvar.raw[:, "MYOD1"].X.toarray().flatten() > 0)
]

In [ ]:
sc.pl.pca(MYOD1_pos_pre)

In [ ]:
sc.pl.pca(MYOD1_pos_post)

### Violin pre post row panel

In [ ]:
data = sc.read_h5ad("/home/alirassolie/Documents/120521_adata_celltype_prevpost_and_markers")
data_filtered = data[data.obs.annotations.str.contains("(?i)sat|myo")]

In [ ]:
genes = ["PAX7", "MYOG", "NCAM1"]

In [ ]:
data_filtered.obs.annotations

In [ ]:
#plt.rcParams['figure.figsize'] = (4,2)
fig, axs = plt.subplots(ncols=1, nrows=len(genes), figsize=(5,8))
axs = axs.flatten()

#with rc_context({'figure.figsize': (4.5, 3)}):

for i, g in enumerate(genes):
    sns.violinplot(
        data_filtered, 
        g, 
        groupby='annotations', 
        stripplot=False, 
        rotation=90, 
        use_raw=False,
        scale="width", 
        ncols=1,
        #save="_celltype_markers.png"
        ax=axs[i],
        xlabel='',
        show=False,
        fill=None
    )
    if i < len(genes)-1: axs[i].tick_params(labelbottom = False)

plt.tight_layout()
plt.show()
# plt.savefig("./FIGURE_1/280521_celltype_markers_violin_9markers.png")

In [ ]:
gene_distrib(data_filtered, genes, s_str="(?i)sat", cluster_column="annotations")

In [ ]:
genes = ["MT1X"]
df = pd.DataFrame(columns=["index", "normcount", "batch", "label", "gene"])
for g in genes:
    df = pd.concat([df, 
               anndata_for_violin(data_filtered, gene=g, toarray=True)])

df = df.loc[:,  ["normcount", "batch", "label", "gene"]]

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(4,4))
axs = axs.flatten()
for i, s_str in enumerate(["sat", "12", "3"]): 
    sns.violinplot(
        data=df[df.label.str.contains(f"(?i){s_str}")],
        x="gene",
        y="normcount",
        hue="batch",
        #cut=0,
        inner=None,
        ax = axs[i],
        scale="width",
        split=True
    )
    axs[i].get_legend().remove()
    axs[i].set_ylim(ymin=0)

plt.tight_layout()
# plt.savefig("/home/alirassolie/Documents/misc/myoblast_prepost_3row_panel.pdf")


### Looking at continuom

In [ ]:
#sc.tl.pca(muscvar, use_highly_variable=True)
sc.pl.pca(muscvar, color=["TNNI1"])
sc.pl.pca(muscvar, color=["TNNI2"])

In [ ]:
pca_df = pd.DataFrame(muscvar.obsm["X_pca"])

In [ ]:
plt.scatter(pca_df[0], pca_df[1])

In [ ]:
sats = muscvar[muscvar.obs.annotations.str.contains("(?i)sat")]

In [ ]:
sats_df = pd.DataFrame(sats.obsm["X_pca"])

In [ ]:
plt.scatter(sats_df[3], sats_df[4], color=map_PCA_to_RGB(sats, "MYF5"))

In [ ]:
def plot_trajectory(anndata, gene, cluster_col="annotations", filter_str="sat", ):
    tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
    df = pd.DataFrame(tmp.obsm["X_pca"])
    df[gene] = tmp[:, gene].X.toarray().flatten()
    df = df[df[gene] > 0]
    fig, ax = plt.subplots(ncols=1)
    ax.scatter(df[0], df[gene])
    #ax.set_ylim(ymin=0.1)

In [ ]:
def plot_trajectory(anndata, gene, cluster_col="annotations", filter_str="sat", ax=None, title="", x_title=""):
    #fig, ax = plt.subplots(ncols=1)
    colors = ["yellow", "brown"]
    for i, b in enumerate(anndata.obs.batch.unique()):
        tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
        tmp = tmp[tmp.obs.batch == b]
        df = pd.DataFrame(tmp.obsm["X_pca"])
        df[gene] = tmp[:, gene].X.toarray().flatten()
        df = df[df[gene] > 0]
        
        ax.scatter(df[0], df[gene], c="black", s=3)
        #ax.scatter(df[1], df[gene], c=colors[i], s=11)
        ax.set_title(title)
        ax.set_xlabel(gene)
    

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(8,5))
axs = axs.flatten()
plot_trajectory(data_filtered, "TNNC2", filter_str="12", ax=axs[0], title="Fast-twitch")

##### pre post for given markers

Myoblast 12 = Fast twitch

Myoblast 3 = Slow twitch

#### boxplot

In [ ]:
def plot_boxplot(anndata, gene, cluster_col="annotations", filter_str="sat"):
    tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
    tmp_pre = tmp[tmp.obs.batch == "0"]
    tmp_post = tmp[tmp.obs.batch == "1"]
    df_pre = pd.DataFrame(tmp_pre.obsm["X_pca"])
    df_post = pd.DataFrame(tmp_post.obsm["X_pca"])
    
    df_pre[gene] = tmp_pre[:, gene].X.toarray().flatten()
    df_post[gene] = tmp_post[:, gene].X.toarray().flatten()
    
    df_pre = df_pre[df_pre[gene] > 0]
    df_post = df_post[df_post[gene] > 0]
    print("pre: ", df_pre[df_pre[gene] > 0][gene].median())
    print("post: ", df_post[df_post[gene] > 0][gene].median())
    
    plt.boxplot([df_pre[gene], df_post[gene]])
    plt.show()

# Redoing the principal curve

In [ ]:
from prinpy.local import CLPCG 
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = sc.read_h5ad("/home/alirassolie/Documents/120521_adata_celltype_prevpost_and_markers")
data_filtered = data[data.obs.annotations.str.contains("(?i)sat|myo")]
muscvar = sc.read_h5ad('/home/alirassolie/Documents/misc/160720_muscvar.h5ad')
muscvar = muscvar[~muscvar.obs.annotations.str.contains("19")]

In [ ]:
cl12 = data_filtered[data_filtered.obs.annotations.str.contains("12|Sat")]
cl3 = data_filtered[data_filtered.obs.annotations.str.contains("3|Sat")]

In [ ]:
cl = CLPCG()
xdata = cl12.obsm['X_pca'].T[0]
ydata = cl12.obsm['X_pca'].T[1]
# cl.fit(xdata, ydata, e_max = .5) 
# cl.plot()       # plots curve, optional axes can be passed
plt.plot(cl.points(xdata, ydata, e_max=0.5).T[0], cl.points(xdata, ydata, e_max=0.5).T[1])
plt.scatter(xdata, ydata)

# Performing linear regression

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import rc_context
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.linear_model import LinearRegression

In [ ]:
data = sc.read_h5ad("/home/alirassolie/Documents/arbete/mint_mobile/120521_adata_celltype_prevpost_and_markers")
data_filtered = data[data.obs.annotations.str.contains("(?i)sat|myo")]
muscvar = sc.read_h5ad('/home/alirassolie/Documents/arbete/mint_mobile/160720_muscvar.h5ad')
muscvar = muscvar[~muscvar.obs.annotations.str.contains("19")]

In [ ]:
adatavar = sc.read_h5ad("/home/alirassolie/Documents/arbete/mint_mobile/003004_230620_adataVar_dpt")

In [ ]:
muscvar0620 = adatavar[adatavar.obs.annotations.str.contains("Sat|Myo")]

In [ ]:
musvar0620sub = muscvar0620[muscvar0620.obs.annotations.str.contains("12|3|Sat")]

In [ ]:
data_filtered_sub = data_filtered[:, muscvar.var_names]

In [ ]:
sc.pl.pca(data_filtered_sub)

In [ ]:
sc.tl.pca(data_filtered_sub)
sc.pl.pca(data_filtered_sub)

In [ ]:
def regressing_anndata_counts(anndata, cluster_col, filter_str, out_df):
    '''Returns a dataframe of gene indices, and the corresponding 
    regression scores for the given genes and the cluster, based on the 
    filter_str provided
    
    Arguments:
    
        anndata:

        gene:

        cluster_col:

        filter_str:

        out_df: This is supposed to have indices of the genes that we wish to run thru. 
    '''
    tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
    
    all_pcs = tmp.obsm["X_pca"].T[0]
    for g in out_df.index:
        # inverting the sign of the pc1
        all_pcs = tmp[:, g].obsm["X_pca"].T[0] * 1
        counts = tmp[:, g].X.toarray().flatten()
        ind = counts > 0
        # X
        all_pcs = all_pcs[ind].reshape(-1, 1)
        
        # y
        counts = counts[ind]#.reshape(-1, 1)
        
        if all_pcs.shape[0] > 30:
            model = LinearRegression().fit(all_pcs, counts)
            score = model.score(all_pcs, counts)
            coeff = model.coef_
            print(coeff)
            out_df.loc[g, "regressor_score"] = score
            out_df.loc[g, "coeff"] = coeff[0]
            
    return out_df

In [ ]:
def index_of_cluster(array_of_clusters, search_by): 
    return np.flatnonzero(np.core.defchararray.find(array_of_clusters, search_by)!=-1)

In [ ]:
# size = len(genes_for_index)
size = 2000

regress_df = pd.DataFrame(
    data = {
        "regressor_score": np.full(size, 0),
        "coeff": np.full(size, 0)
    },
    index = muscvar.var[muscvar.var.highly_variable].index.values
)


# regressing_anndata_counts(data_filtered, "annotations", "12", out_df = regress_df)
# regressing_anndata_counts(data_filtered, "annotations", "12", out_df = regress_df)

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(8,5))
axs = axs.flatten()
# plot_trajectory(data_filtered, "CKM", filter_str="12", ax=axs[0], title="Fast-twitch")
plot_trajectory(data_filtered, "TNNC2", filter_str="12", ax=axs[0], title="Fast-twitch")
plot_trajectory(data_filtered, "MYOD1", filter_str="12", ax=axs[1], title="Fast-twitch")
plot_trajectory(data_filtered, "MYF6", filter_str="12", ax=axs[2], title="Fast-twitch")

plot_trajectory(data_filtered, "TNNC1", filter_str="3", ax=axs[3], title="Slow-twitch")
plot_trajectory(data_filtered, "MYOD1", filter_str="3", ax=axs[4], title="Slow-twitch")
plot_trajectory(data_filtered, "MYF6", filter_str="3", ax=axs[5], title="Slow-twitch")

plt.tight_layout()
#plt.savefig("/home/alirassolie/Documents/misc/myoblast_expression_scatterplot_along_pc.pdf")

In [ ]:
def trajectory_data(anndata, gene, cluster_col="annotations", filter_str="sat", ax=None, title="", x_title=""):
    tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
    df = pd.DataFrame(tmp.obsm["X_pca"])
    df[gene] = tmp[:, gene].X.toarray().flatten()
    #df = df[df[gene] > 0]
    return df    

In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt


In [ ]:
%matplotlib inline

def plot_ci_manual(t, s_err, n, x, x2, y2, ax=None):
    """Return an axes of confidence bands using a simple approach.

    Notes
    -----
    .. math:: \left| \: \hat{\mu}_{y|x0} - \mu_{y|x0} \: \right| \; \leq \; T_{n-2}^{.975} \; \hat{\sigma} \; \sqrt{\frac{1}{n}+\frac{(x_0-\bar{x})^2}{\sum_{i=1}^n{(x_i-\bar{x})^2}}}
    .. math:: \hat{\sigma} = \sqrt{\sum_{i=1}^n{\frac{(y_i-\hat{y})^2}{n-2}}}

    References
    ----------
    .. [1] M. Duarte.  "Curve fitting," Jupyter Notebook.
       http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/CurveFitting.ipynb

    """
    if ax is None:
        ax = plt.gca()

    ci = t * s_err * np.sqrt(1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))
    ax.fill_between(x2, y2 + ci, y2 - ci)

    return ax


def plot_ci_bootstrap(xs, ys, resid, nboot=500, ax=None):
    """Return an axes of confidence bands using a bootstrap approach.

    Notes
    -----
    The bootstrap approach iteratively resampling residuals.
    It plots `nboot` number of straight lines and outlines the shape of a band.
    The density of overlapping lines indicates improved confidence.

    Returns
    -------
    ax : axes
        - Cluster of lines
        - Upper and Lower bounds (high and low) (optional)  Note: sensitive to outliers

    References
    ----------
    .. [1] J. Stults. "Visualizing Confidence Intervals", Various Consequences.
       http://www.variousconsequences.com/2010/02/visualizing-confidence-intervals.html

    """ 
    if ax is None:
        ax = plt.gca()

    bootindex = sp.random.randint

    for _ in range(nboot):
        resamp_resid = resid[bootindex(0, len(resid) - 1, len(resid))]
        # Make coeffs of for polys
        pc = sp.polyfit(xs, ys + resamp_resid, 1)                   
        # Plot bootstrap cluster
        ax.plot(xs, sp.polyval(pc, xs), "b-", linewidth=2, alpha=3.0 / float(nboot))

    return ax

In [ ]:
# Modeling with Numpy
def equation(a, b):
    """Return a 1D polynomial."""
    return np.polyval(a, b) 


# Plotting --------------------------------------------------------------------

def plot_with_confidence(
        ax, x, y, weights, heights, score, title="", label="",
        
):
    p, cov = np.polyfit(x, y, 1, cov=True)                     # parameters and covariance from of the fit of 1-D polynom.
    y_model = equation(p, x)                                   # model using the fit parameters; NOTE: parameters here are coefficients
    print(p, cov)

    # Statistics
    n = weights.size                                           # number of observations
    m = p.size                                                 # number of parameters
    dof = n - m                                                # degrees of freedom
    t = stats.t.ppf(0.975, n - m)                              # used for CI and PI bands

    # Estimates of Error in Data/Model
    resid = y - y_model                           
    chi2 = np.sum((resid / y_model)**2)                        # chi-squared; estimates error in data
    chi2_red = chi2 / dof                                      # reduced chi-squared; measures goodness of fit
    s_err = np.sqrt(np.sum(resid**2) / dof)                    # standard deviation of the error

    # Data
    ax.plot(
        x, y, "o", color="yellow", 
        markersize=8, 
        markeredgewidth=1, 
        markeredgecolor="black",
        markerfacecolor="None",
        alpha=0.5
    )

    # Fit
    ax.plot(x, y_model, "-", color="0.1", linewidth=1.5, alpha=0.5, label="Fit")  

    x2 = np.linspace(np.min(x), np.max(x), 100)
    y2 = equation(p, x2)

    # Confidence Interval (select one)
    plot_ci_manual(t, s_err, n, x, x2, y2, ax=ax)
    #plot_ci_bootstrap(x, y, resid, ax=ax)

    # Prediction Interval
    pi = t * s_err * np.sqrt(1 + 1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))   
    ax.fill_between(x2, y2 + pi, y2 - pi, color="None", linestyle="--")
    ax.plot(x2, y2 - pi, "--", color="0.5", label="95% Prediction Limits")
    ax.plot(x2, y2 + pi, "--", color="0.5")


    # Figure Modifications --------------------------------------------------------
    # Borders
    ax.spines["top"].set_color("0.5")
    ax.spines["bottom"].set_color("0.5")
    ax.spines["left"].set_color("0.5")
    ax.spines["right"].set_color("0.5")
    ax.get_xaxis().set_tick_params(direction="out")
    ax.get_yaxis().set_tick_params(direction="out")
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left() 

    # Labels
    ax.set_title(f"{title}, R^2={score}", fontsize="14")
    ax.set_xlabel(label)
    
    plt.xlim(np.min(x) - 1, np.max(x) + 1)

    # Custom legend
    handles, labels = ax.get_legend_handles_labels()
    display = (0, 1)



In [ ]:
# Modeling with Numpy
def equation(a, b):
    """Return a 1D polynomial."""
    return np.polyval(a, b) 


# Plotting --------------------------------------------------------------------

def plot_with_confidence(
        ax, x, y, weights, heights, score, title="", label="", cl="line1"
    ):
    p, cov = np.polyfit(x, y, 1, cov=True)   # parameters and covariance from of the fit of 1-D polynom.
    y_model = equation(p, x)                 # model using the fit parameters; parameters here are coefficients

    # Statistics
    n = weights.size                                           # number of observations
    m = p.size                                                 # number of parameters
    dof = n - m                                                # degrees of freedom
    t = stats.t.ppf(0.975, n - m)                              # used for CI and PI bands

    # Estimates of Error in Data/Model
    resid = y - y_model                           
    chi2 = np.sum((resid / y_model)**2)                        # chi-squared; estimates error in data
    chi2_red = chi2 / dof                                      # reduced chi-squared; measures goodness of fit
    s_err = np.sqrt(np.sum(resid**2) / dof)                    # standard deviation of the error
    print(cl)
    if cl=="line2":
        color="yellow"
        edge="black"
    else:
        color="blue"
        edge="white"
    # Data
    ax.plot(
        x, y, "o", color=color, 
        markersize=8, 
        markeredgewidth=1, 
        markeredgecolor=edge,
        # markerfacecolor="None",
        #alpha=0.5
    )

    # Fit
    ax.plot(x, y_model, "-", color="0.1", linewidth=1.5, alpha=0.5, label="Fit")  

    x2 = np.linspace(np.min(x), np.max(x), 100)
    y2 = equation(p, x2)

    # Confidence Interval (select one)
    plot_ci_manual(t, s_err, n, x, x2, y2, ax=ax)
    #plot_ci_bootstrap(x, y, resid, ax=ax)

    # Prediction Interval
    pi = t * s_err * np.sqrt(1 + 1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))   
    ax.fill_between(x2, y2 + pi, y2 - pi, color="None", linestyle="--")
    ax.plot(x2, y2 - pi, "--", color="0.5", label="95% Prediction Limits")
    ax.plot(x2, y2 + pi, "--", color="0.5")


    # Figure Modifications --------------------------------------------------------
    # Borders
    ax.spines["top"].set_color("0.5")
    ax.spines["bottom"].set_color("0.5")
    ax.spines["left"].set_color("0.5")
    ax.spines["right"].set_color("0.5")
    ax.get_xaxis().set_tick_params(direction="out")
    ax.get_yaxis().set_tick_params(direction="out")
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left() 

    # Labels
    ax.set_title(f"{title}, R^2={score}", fontsize="14")
    ax.set_xlabel(label)
    
    plt.xlim(np.min(x) - 1, np.max(x) + 1)

    # Custom legend
    handles, labels = ax.get_legend_handles_labels()
    display = (0, 1)



In [ ]:

size = 5
regress_df_selected_genes = pd.DataFrame(
    data = {
        "regressor_score": np.full(size, 0),
        "coeff": np.full(size, 0)
    },
    index = ["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYF6"]
)

regressors_12_selected = regressing_anndata_counts(data_filtered, "annotations", "12", regress_df_selected_genes)

regress_df_selected_genes = pd.DataFrame(
    data = {
        "regressor_score": np.full(size, 0),
        "coeff": np.full(size, 0)
    },
    index = ["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYF6"]
)
regressors_3_selected = regressing_anndata_counts(data_filtered, "annotations", "3", regress_df_selected_genes)
print(regressors_12_selected)
print(regressors_3_selected)

In [ ]:
scores = {"12":
       
{"TNNC2": 0.316457, 
"MYOD1": 0.129255, 
"MYF6":  0.224659, 
"TNNC1": 0.050856, 
"MYF6":  0.224659 
},
"3":
       
{"TNNC2": 0.046891, 
"MYOD1": 0.385308, 
"MYF6":  0.289449, 
"TNNC1": 0.281850, 
"MYF6":  0.289449 }
}

In [ ]:

fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10, 5))
axs = axs.flatten()
for i, (gene, cl) in enumerate(zip(["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYOD1", "MYF6"], ["12", "12", "12", "3", "3", "3"])):
    if cl == "12": tit = "Fast-twitch"
    elif cl == "3": tit = "Slow-twitch"
    rn_df = trajectory_data(data_filtered, gene, filter_str=cl, ax=axs[0], title=tit)
    rn_df = rn_df.loc[:, [0, gene]]
    rn_df = rn_df[rn_df[gene] > 0]

    # flipping the sign of pc1
    #rn_df.loc[:, 0] = rn_df.loc[:, 0] * -1
    
    # Computations ----------------------------------------------------------------
    # Raw Data
    weights = np.array(rn_df[gene])
    heights = np.array(rn_df[0])

    x = heights
    y = weights
    if np.min(x) < 0:
        x = np.add(x, np.abs(np.min(x)))
    plot_with_confidence(ax=axs[i], x=x, y=y, weights=weights, heights=heights, title=tit, label=gene,
                        score=scores[cl][gene]
                    )

# Save Figure
plt.tight_layout()

#plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/myoblast_scatter_linear_regression.pdf")

In [ ]:
x1 = np.array([-1, -2, -3, 2, 3, 5])

In [ ]:
x1 + np.abs(np.min(x1))

In [ ]:
sc.tl.pca(data_filtered)
sc.pl.pca(data_filtered)

# KDE

In [ ]:
fig, ax = plt.subplots()
df = pd.DataFrame({
    'x': muscvar[(muscvar.obs.annotations.str.contains("12")) & (muscvar.obs.batch == "0")].obsm["X_pca"].T[0]*-1,
})

df.plot.kde(ax=ax)
df = pd.DataFrame({
'y': muscvar[(muscvar.obs.annotations.str.contains("12")) & (muscvar.obs.batch == "1")].obsm["X_pca"].T[0]*-1
})

df.plot.kde(ax=ax)
    

In [ ]:
fig, ax = plt.subplots()
df = pd.DataFrame({
    'x': muscvar[(muscvar.obs.annotations.str.contains("3")) & (muscvar.obs.batch == "0")].obsm["X_pca"].T[0]*-1,
})

df.plot.kde(ax=ax)
df = pd.DataFrame({
'y': muscvar[(muscvar.obs.annotations.str.contains("3")) & (muscvar.obs.batch == "1")].obsm["X_pca"].T[0]*-1
})

df.plot.kde(ax=ax)
    

In [ ]:
plt.scatter(
    muscvar[(muscvar.obs.annotations.str.contains("12|3")) & (muscvar.obs.batch == "0")].obsm["X_pca"].T[0]*-1,
    muscvar[(muscvar.obs.annotations.str.contains("12|3")) & (muscvar.obs.batch == "0")].obsm["X_pca"].T[1],
)

plt.scatter(
    muscvar[(muscvar.obs.annotations.str.contains("12|3")) & (muscvar.obs.batch == "1")].obsm["X_pca"].T[0]*-1,
    muscvar[(muscvar.obs.annotations.str.contains("12|3")) & (muscvar.obs.batch == "1")].obsm["X_pca"].T[1],
)

In [ ]:
fig, ax = plt.subplots()
df = pd.DataFrame({
    'pre': data_filtered[(data_filtered.obs.annotations.str.contains("3")) & (data_filtered.obs.batch == "0")].obsm["X_pca"].T[0]*-1,
})

df.plot.kde(ax=ax)
df = pd.DataFrame({
'post': data_filtered[(data_filtered.obs.annotations.str.contains("3")) & (data_filtered.obs.batch == "1")].obsm["X_pca"].T[0]*-1
})

df.plot.kde(ax=ax)
    

# Cumulative Distribution, ECDF

In [ ]:
def ECDF(one_d: np.array, sort:bool=False, perc:bool=False, ax=None):
    """ECDF function will produce a scatterplot and return 
    the resulting np.array of the sorted data, with the ECDF
    calculated

    Arguments:
        
        one_d: a np.array, a vector containing the data to be ECDF'd

        sort: boolean, if the input vector should be sorted before
        processed

        perc: boolean, if the ECDF should present the cumulative sum 
        as values of a quotient range(0,1)
    """

    if sort: one_d = one_d[one_d.argsort()];
    one_d_shifted = one_d + np.abs(min(one_d));
    cum = np.full(one_d.shape[0], 0.0);
    total = sum(one_d_shifted);

    if perc:
        for i in range(cum.shape[0]):
            cum[i] = sum(one_d_shifted[:i]) / total;
    else:
        for i in range(cum.shape[0]):
            cum[i] = sum(one_d_shifted[:i]);
    if not ax:
        plt.scatter(one_d_shifted, cum);
        plt.show();
    elif ax:
        ax.scatter(one_d_shifted, cum);
    return (one_d_shifted, cum)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# some fake data
data = np.random.randn(1000)
# evaluate the histogram
values, base = np.histogram(data, bins=40)
#evaluate the cumulative
cumulative = np.cumsum(values)
# plot the cumulative function
plt.plot(base[:-1], cumulative, c='blue')
#plot the survival function
plt.plot(base[:-1], len(data)-cumulative, c='green')

plt.show()

In [ ]:
cl12 = muscvar[muscvar.obs.annotations.str.contains("12")]
cl3 = muscvar[muscvar.obs.annotations.str.contains("3")]

batch_12, batch_3 = [], []

for i in range(2): 
    batch_12.append(cl12[cl12.obs.batch == f"{i}"])
    batch_3.append(cl3[cl3.obs.batch == f"{i}"])

In [ ]:
cl12_pca = pd.DataFrame(cl12.obsm["X_pca"]).loc[:, :1]
cl12_pca["batch"] = cl12.obs.batch.values

cl3_pca = pd.DataFrame(cl3.obsm["X_pca"]).loc[:, :1]
cl3_pca["batch"] = cl3.obs.batch.values

Add boxplots

In [ ]:


fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(15,6))
# axs = axs.flatten()

for i in range(2):
    pc_df = batch_12[i].obsm["X_pca"].T[0]*-1 + np.sqrt(min(cl12.obsm["X_pca"].T[0]*-1)**2)
    values, base = np.histogram(pc_df, bins=20)
    cumulative = np.cumsum(values)
    axs[0].plot(base[:-1], np.divide(cumulative, cumulative[-1]), linewidth=1)
    axs[0].scatter(base[:-1], np.divide(cumulative, cumulative[-1]), linewidth=1)


for i in range(2):
    pc_df = batch_3[i].obsm["X_pca"].T[0]*-1 + np.sqrt(min(cl3.obsm["X_pca"].T[0]*-1)**2)
    values, base = np.histogram(pc_df, bins=20)
    cumulative = np.cumsum(values)
    axs[1].plot(base[:-1], np.divide(cumulative, cumulative[-1]))
    axs[1].scatter(base[:-1], np.divide(cumulative, cumulative[-1]))

axs[0].set_xlabel("Slow-twitch")    
axs[1].set_xlabel("Fast-twitch")

plt.savefig("/home/alirassolie/Documents/misc/cumsum_fast_slow.pdf")

plt.tight_layout()

In [ ]:
cl12_pca = pd.DataFrame(cl12.obsm["X_pca"]).loc[:, :1]
cl12_pca["batch"] = cl12.obs.batch.values

In [ ]:
cl12_pca.loc[:, 'batch'] = cl12_pca.batch.astype(int)
cl3_pca.loc[:, 'batch'] = cl3_pca.batch.astype(int)


In [ ]:
fig, axs = plt.subplots(ncols=2, figsize = (4,2))


batch = lambda df, i: df[df.batch == 1]

cl12_pca.loc[:, 0] = cl12_pca.loc[:, 0] + np.sqrt(min(cl12_pca[0]) ** 2)
cl3_pca.loc[:, 0] = cl3_pca.loc[:, 0] + np.sqrt(min(cl3_pca[0]) ** 2)


sns.boxplot(data=cl12_pca, y=0, x="batch", ax=axs[0], showfliers = False)
sns.boxplot(data=cl3_pca, y=0, x="batch", ax=axs[1], showfliers = False)

plt.tight_layout()
plt.savefig("/home/alirassolie/Documents/misc/boxplot_fastslow_pca1.pdf")

In [ ]:
fig, axs = plt.subplots(ncols=1)
ECDF(batch_12[0].obsm["X_pca"].T[0], sort=True, perc=True, ax=axs)
ECDF(batch_12[1].obsm["X_pca"].T[0], sort=True, perc=True, ax=axs)

### Outputting a data-matrix

In [ ]:
data_matrix = pd.DataFrame(data_filtered.X.toarray(), columns=data_filtered.var_names, index=data_filtered.obs.index)
data_matrix["annotation"] = data_filtered.obs.annotations
data_matrix["batch"] = data_filtered.obs.batch
data_matrix.to_csv("/home/alirassolie/Documents/misc/data_matrix_myoblasts.csv")


# Redoing with slingshot coordinates

In [ ]:
from collections import Counter
import random
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import scanpy as sc
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import rc_context
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
data = sc.read_h5ad("/home/alirassolie/Documents/arbete/mint_mobile/120521_adata_celltype_prevpost_and_markers")
data_filtered = data[data.obs.annotations.str.contains("(?i)sat|myo")]
muscvar = sc.read_h5ad('/home/alirassolie/Documents/arbete/mint_mobile/160720_muscvar.h5ad')
muscvar = muscvar[~muscvar.obs.annotations.str.contains("19")]
pt = pd.read_csv("/home/alirassolie/Documents/prwork/klinfys/data/pseudotime_080722.csv")
data_filtered = data_filtered[~data_filtered.obs.annotations.str.contains("(?i)sat")]
pre = data_filtered[data_filtered.obs.batch=="0"]
post = data_filtered[data_filtered.obs.batch=="1"]
pt_filtered = pt[pt.cells.isin(data_filtered.obs.index.values)]
pt_filtered.loc[pt_filtered.line1.notnull(), "cells"]
pt_filtered.loc[:, "line1"] = pt_filtered.loc[:, "line1"] - pt_filtered.loc[:, "line1"].min()
pt_filtered.loc[:, "line2"] = pt_filtered.loc[:, "line2"] - pt_filtered.loc[:, "line2"].min()
pt_pre = pt_filtered[pt_filtered.cells.isin(pre.obs.index.values)]
pt_post = pt_filtered[pt_filtered.cells.isin(post.obs.index.values)]

In [ ]:
def plot_trajectory(anndata, gene, cluster_col="annotations", filter_str="sat", ax=None, title="", x_title=""):
    #fig, ax = plt.subplots(ncols=1)
    colors = ["yellow", "brown"]
    for i, b in enumerate(anndata.obs.batch.unique()):
        tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
        tmp = tmp[tmp.obs.batch == b]
        df = pd.DataFrame(tmp.obsm["X_pca"])
        df[gene] = tmp[:, gene].X.toarray().flatten()
        df = df[df[gene] > 0]
        
        ax.scatter(df[0], df[gene], c="black", s=3)
        #ax.scatter(df[1], df[gene], c=colors[i], s=11)
        ax.set_title(title)
        ax.set_xlabel(gene)
    

In [ ]:
def test_fidelity():
    for i in range(100):
        i = random.randint(0,2000)
        print(data_filtered[pt[pt.line1.notnull()].cells][:, "TNNI1"].obs.index.values[i] == pt[pt.line1.notnull()].cells.values[i])

In [ ]:
def scatter_pt(anndata, pseudotime, gene="TNNI1", col="line1", **kwargs):
    pseudotime[gene] = anndata[pseudotime.cells][:, gene].X.toarray().flatten()
    tm = pseudotime[pseudotime[gene] != 0]
    tm = tm[tm[col].notnull()]
    plt.scatter(tm[col], tm[gene], **kwargs)
    plt.show()

In [ ]:
def return_gene_with_pt(anndata, pseudotime, gene="TNNI1", col="line1", **kwargs):
    pseudotime[gene] = anndata[pseudotime.cells][:, gene].X.toarray().flatten()
    pseudotime["PC0"] = anndata[pseudotime.cells].obsm["X_pca"][:, 0]
    pseudotime["PC1"] = anndata[pseudotime.cells].obsm["X_pca"][:, 1]
    tm = pseudotime[pseudotime[gene] != 0]
    tm = tm[tm[col].notnull()]
    return tm

In [ ]:
%matplotlib inline

def plot_ci_manual(t, s_err, n, x, x2, y2, ax=None):
    """Return an axes of confidence bands using a simple approach.

    Notes
    -----
    .. math:: \left| \: \hat{\mu}_{y|x0} - \mu_{y|x0} \: \right| \; \leq \; T_{n-2}^{.975} \; \hat{\sigma} \; \sqrt{\frac{1}{n}+\frac{(x_0-\bar{x})^2}{\sum_{i=1}^n{(x_i-\bar{x})^2}}}
    .. math:: \hat{\sigma} = \sqrt{\sum_{i=1}^n{\frac{(y_i-\hat{y})^2}{n-2}}}

    References
    ----------
    .. [1] M. Duarte.  "Curve fitting," Jupyter Notebook.
       http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/CurveFitting.ipynb

    """
    if ax is None:
        ax = plt.gca()

    ci = t * s_err * np.sqrt(1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))
    ax.fill_between(x2, y2 + ci, y2 - ci)

    return ax


def plot_ci_bootstrap(xs, ys, resid, nboot=500, ax=None):
    """Return an axes of confidence bands using a bootstrap approach.

    Notes
    -----
    The bootstrap approach iteratively resampling residuals.
    It plots `nboot` number of straight lines and outlines the shape of a band.
    The density of overlapping lines indicates improved confidence.

    Returns
    -------
    ax : axes
        - Cluster of lines
        - Upper and Lower bounds (high and low) (optional)  Note: sensitive to outliers

    References
    ----------
    .. [1] J. Stults. "Visualizing Confidence Intervals", Various Consequences.
       http://www.variousconsequences.com/2010/02/visualizing-confidence-intervals.html

    """ 
    if ax is None:
        ax = plt.gca()

    bootindex = sp.random.randint

    for _ in range(nboot):
        resamp_resid = resid[bootindex(0, len(resid) - 1, len(resid))]
        # Make coeffs of for polys
        pc = sp.polyfit(xs, ys + resamp_resid, 1)                   
        # Plot bootstrap cluster
        ax.plot(xs, sp.polyval(pc, xs), "b-", linewidth=2, alpha=3.0 / float(nboot))

    return ax

In [ ]:
# Modeling with Numpy
def equation(a, b):
    """Return a 1D polynomial."""
    return np.polyval(a, b) 


# Plotting --------------------------------------------------------------------

def plot_with_confidence(
        ax, x, y, weights, heights, score, title="", label="", cl="line1"
    ):
    p, cov = np.polyfit(x, y, 1, cov=True)   # parameters and covariance from of the fit of 1-D polynom.
    y_model = equation(p, x)                 # model using the fit parameters; parameters here are coefficients

    # Statistics
    n = weights.size                                           # number of observations
    m = p.size                                                 # number of parameters
    dof = n - m                                                # degrees of freedom
    t = stats.t.ppf(0.975, n - m)                              # used for CI and PI bands

    # Estimates of Error in Data/Model
    resid = y - y_model                           
    chi2 = np.sum((resid / y_model)**2)                        # chi-squared; estimates error in data
    chi2_red = chi2 / dof                                      # reduced chi-squared; measures goodness of fit
    s_err = np.sqrt(np.sum(resid**2) / dof)                    # standard deviation of the error
    print(cl)
    if cl=="line2":
        color="yellow"
        edge="black"
    else:
        color="blue"
        edge="white"
    # Data
    ax.scatter(
        x, y, c=x, cmap="viridis",
    )

    # Fit
    ax.plot(x, y_model, "-", color="0.1", linewidth=1.5, alpha=0.5, label="Fit")  

    x2 = np.linspace(np.min(x), np.max(x), 100)
    y2 = equation(p, x2)

    # Confidence Interval (select one)
    plot_ci_manual(t, s_err, n, x, x2, y2, ax=ax)
    #plot_ci_bootstrap(x, y, resid, ax=ax)

    # Prediction Interval
    pi = t * s_err * np.sqrt(1 + 1/n + (x2 - np.mean(x))**2 / np.sum((x - np.mean(x))**2))   
    ax.fill_between(x2, y2 + pi, y2 - pi, color="None", linestyle="--")
    ax.plot(x2, y2 - pi, "--", color="0.5", label="95% Prediction Limits")
    ax.plot(x2, y2 + pi, "--", color="0.5")


    # Figure Modifications --------------------------------------------------------
    # Borders
    ax.spines["top"].set_color("0.5")
    ax.spines["bottom"].set_color("0.5")
    ax.spines["left"].set_color("0.5")
    ax.spines["right"].set_color("0.5")
    ax.get_xaxis().set_tick_params(direction="out")
    ax.get_yaxis().set_tick_params(direction="out")
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left() 

    # Labels
    ax.set_title(f"{title}, R^2={score}", fontsize="14")
    ax.set_xlabel(label)
    
    plt.xlim(np.min(x) - 1, np.max(x) + 1)

    # Custom legend
    handles, labels = ax.get_legend_handles_labels()
    display = (0, 1)



In [ ]:
def regress(x,y):
    model = LinearRegression().fit(x, y)
    score = model.score(x, y)
    coeff = model.coef_
    return score

#### baseline + post

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20, 8))
axs = axs.flatten()
for i, (gene, cl) in enumerate(zip(["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYOD1", "MYF6"], ["line1", "line1", "line1", "line2", "line2", "line2"])):
    dt = return_gene_with_pt(data_filtered, pt_filtered, gene=gene, col=cl)
    dt.to_csv(f"/home/alirassolie/Documents/arbete/mint_mobile/linreg_data_{gene}_{cl}.csv")
    x = dt[cl].values
    y = dt[gene].values
    score = regress(x.reshape(x.shape[0], 1), y)
#     print("r2 score ", score) 
#     print("np score ", np.corrcoef(x,y)**2)
    plot_with_confidence(
                        ax=axs[i], 
                        x=x, 
                        y=y, 
                        weights=x,
                        heights=y, 
                        title=gene, 
                        label=gene, 
                        score=f"{regress(x.reshape(x.shape[0], 1), y)}"[:4],
                        cl=cl
                        )
# plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/linearregression_slingshotpseudotime_preAndPost.png")
plt.tight_layout()
# plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/linearregression_slingshotpseudotime_preAndPost_zeroed.pdf")


#### baseline

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20, 10))
axs = axs.flatten()
for i, (gene, cl) in enumerate(zip(["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYOD1", "MYF6"], ["line1", "line1", "line1", "line2", "line2", "line2"])):
    dt = return_gene_with_pt(pre, pt_pre, gene=gene, col=cl)
    x = dt[cl].values
    y = dt[gene].values
    score = regress(x.reshape(x.shape[0], 1), y)
    print("r2 score ", score) 
    print("np score ", np.corrcoef(x,y)**2)
    plot_with_confidence(ax=axs[i], x=x,y=y, weights=x,heights=y, 
                     title="", label=gene, score=f"{regress(x.reshape(x.shape[0], 1), y)}"[:4])

#### post

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(20, 10))
axs = axs.flatten()
for i, (gene, cl) in enumerate(zip(["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYOD1", "MYF6"], ["line1", "line1", "line1", "line2", "line2", "line2"])):
    dt = return_gene_with_pt(post, pt_post, gene=gene, col=cl)
    x = dt[cl].values
    y = dt[gene].values
    score = regress(x.reshape(x.shape[0], 1), y)
    print("r2 score ", score) 
    print("np score ", np.corrcoef(x,y)**2)
    plot_with_confidence(ax=axs[i], x=x,y=y, weights=x,heights=y, 
                     title="", label=gene, score=f"{regress(x.reshape(x.shape[0], 1), y)}"[:4])

##### Pseudotime as color

In [ ]:
def trajectory_data(anndata, gene, cluster_col="annotations", filter_str="sat", ax=None, title="", x_title="", pt=None):
    tmp = anndata[anndata.obs[cluster_col].str.contains(f"(?i){filter_str}")]
    df = pd.DataFrame(tmp.obsm["X_pca"])
    df[gene] = tmp[:, gene].X.toarray().flatten()
    return df, tmp

In [ ]:
return_gene_with_pt(data_filtered, pt_filtered, gene=gene, col="line1")

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=2, figsize=(10, 5))
axs = axs.flatten()
for i, (gene, cl) in enumerate(zip(["TNNC2", "MYOD1", "MYF6", "TNNC1", "MYOD1", "MYF6"], ["line1", "line1", "line1", "line2", "line2", "line2"])):
    if cl == "12": tit = "Fast-twitch"
    elif cl == "3": tit = "Slow-twitch"
    pt_df_o = return_gene_with_pt(data_filtered, pt_filtered, gene=gene, col="line2")
    # rn_df = trajectory_data(data_filtered, gene, filter_str=cl, ax=axs[0], title=tit)
    # pt_df = pt_df.loc[:, [0, gene]]
    pt_df = pt_df_o[pt_df_o[gene] > 0]

    # flipping the sign of pc1
    #pt_df.loc[:, "PC0"] = pt_df.loc[:, "PC0"] * -1
    
    # Computations ----------------------------------------------------------------
    # Raw Data
    weights = np.array(pt_df[gene])
    heights = np.array(pt_df["PC0"])

    x = heights
    y = weights
    if np.min(x) < 0:
        x = np.add(x, np.abs(np.min(x)))
    plot_with_confidence(ax=axs[i], x=x, y=y, weights=weights, heights=heights, title="", label=gene,
                        score=scores[cl][gene]
                    )

# Save Figure
plt.tight_layout()

#plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/myoblast_scatter_linear_regression.pdf")

In [ ]:
return_gene_with_pt(data_filtered, pt_filtered, gene=gene, col="line1").loc[:, gene]

### Redoing the ECDFs after review comments round 2

In [ ]:
def ECDF(one_d: np.array, sort:bool=False, perc:bool=False, ax=None):
    """ECDF function will produce a scatterplot and return 
    the resulting np.array of the sorted data, with the ECDF
    calculated

    Arguments:
        
        one_d: a np.array, a vector containing the data to be ECDF'd

        sort: boolean, if the input vector should be sorted before
        processed

        perc: boolean, if the ECDF should present the cumulative sum 
        as values of a quotient range(0,1)
    """

    if sort: one_d = one_d[one_d.argsort()];
    one_d_shifted = one_d + np.abs(min(one_d));
    cum = np.full(one_d.shape[0], 0.0);
    total = sum(one_d_shifted);

    if perc:
        for i in range(cum.shape[0]):
            cum[i] = sum(one_d_shifted[:i]) / total;
    else:
        for i in range(cum.shape[0]):
            cum[i] = sum(one_d_shifted[:i]);
    if not ax:
        plt.scatter(one_d_shifted, cum);
        plt.show();
    elif ax:
        ax.scatter(one_d_shifted, cum);
    return (one_d_shifted, cum)

In [ ]:
def ECDF_histo(one_d: np.array, sort:bool=False, perc:bool=False, ax=None):
    """ECDF function will produce a scatterplot and return 
    the resulting np.array of the sorted data, with the ECDF
    calculated

    Arguments:
        
        one_d: a np.array, a vector containing the data to be ECDF'd

        sort: boolean, if the input vector should be sorted before
        processed

        perc: boolean, if the ECDF should present the cumulative sum 
        as values of a quotient range(0,1)
    """

    if sort: one_d = one_d[one_d.argsort()];
    one_d_shifted = one_d #+ np.abs(min(one_d));
    hi_vals, base = np.histogram(one_d_shifted, bins=20)
    cum = np.cumsum(hi_vals)
    cum = np.divide(cum, cum[-1])
    print(len(cum), len(base))
    if not ax:
        #plt.scatter(base[:-1], cum);
        plt.plot(base[:-1], cum);
        plt.show();
    elif ax:
        #ax.scatter(base[:-1], cum);
        ax.plot(base[:-1], cum);
    return (one_d_shifted, cum)

***LINE1*** FAST Twitch..?

In [ ]:
fig, axs = plt.subplots(ncols=1)
line1_pre_cum = ECDF(pt_pre.loc[pt_pre.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs)
line1_post_cum = ECDF(pt_post.loc[pt_post.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs)

In [ ]:
fig, axs = plt.subplots(ncols=1)
_ = ECDF_histo(pt_pre.loc[pt_pre.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs)
_ = ECDF_histo(pt_post.loc[pt_post.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs)

***LINE2*** SLOW Twitch..?

In [ ]:
fig, axs = plt.subplots(ncols=1)
line2_pre_cum = ECDF(pt_pre.loc[pt_pre.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs)
line2_post_cum = ECDF(pt_post.loc[pt_post.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs)

In [ ]:
fig, axs = plt.subplots(ncols=1)
_ = ECDF_histo(pt_pre.loc[pt_pre.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs)
_ = ECDF_histo(pt_post.loc[pt_post.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs)

In [ ]:
sns.boxplot(data=[pt_pre.line1, pt_post.line1])
plt.show()

In [ ]:
sns.boxplot(data=[pt_pre.line2, pt_post.line2])

In [ ]:
writer = pd.ExcelWriter("/home/alirassolie/Documents/arbete/mint_mobile/finalreview/Supplement_10_Figure_5_boxplot_290922.xlsx", engine="xlsxwriter")
pt.to_excel(writer, sheet_name="Figure_5_boxplot")
writer.save()

##### Zeroing the dataset

In [ ]:
def find_min(df1, df2, col):
    return np.min([df1.loc[:, col].min(), df2.loc[:, col].min()])
    

In [ ]:
line1_const = find_min(pt_pre, pt_post, "line1")
line2_const = find_min(pt_pre, pt_post, "line2")

In [ ]:
pt_pre.loc[pt_pre.line1.notnull(), "line1"] = pt_pre.loc[pt_pre.loc[:, "line1"].notnull(), "line1"] - line1_const

In [ ]:
pt_pre.loc[pt_pre.loc[:, "line2"].notnull(), "line2"] = pt_pre.loc[pt_pre.loc[:, "line2"].notnull(), "line2"] - line2_const

In [ ]:
pt_post.loc[pt_post.loc[:, "line2"].notnull(), "line2"] = pt_post.loc[pt_post.loc[:, "line2"].notnull(), "line2"] - line2_const

In [ ]:
pt_post.loc[pt_post.loc[:, "line1"].notnull(), "line1"] = pt_post.loc[pt_post.loc[:, "line1"].notnull(), "line1"] - line1_const

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(14,8))#, gridspec_kw={'width_ratios': [3, 3, 1, 1]})
axs = axs.flatten()
_ = ECDF_histo(pt_pre.loc[pt_pre.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs[0])
_ = ECDF_histo(pt_post.loc[pt_post.line1.notnull(), "line1"].to_numpy(), sort=True, perc=True, ax=axs[0])

_ = ECDF_histo(pt_pre.loc[pt_pre.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs[1])
_ = ECDF_histo(pt_post.loc[pt_post.line2.notnull(), "line2"].to_numpy(), sort=True, perc=True, ax=axs[1])

sns.boxplot(data=[pt_pre.line1, pt_post.line1], ax=axs[2])
sns.boxplot(data=[pt_pre.line2, pt_post.line2], ax=axs[3])

axs[0].title.set_text("Line1")
axs[1].title.set_text("Line2")
# plt.savefig("/home/alirassolie/Documents/arbete/mint_mobile/ECDF_withboxplot_withslingshotdata_zeroed.pdf")

#### Labelling the clusters and different differentiation stages

In [ ]:
dt = return_gene_with_pt(data_filtered, pt_filtered, gene="MYH3", col=cl)
dt

In [ ]:
data_filtered.X = data_filtered.raw[:, data_filtered.var_names].X



In [ ]:
pc1 = muscvar.obsm["X_pca"][:, 0]
pc2 = muscvar.obsm["X_pca"][:, 1]

In [ ]:
myh7 = data_filtered[:, "MYH7"].X.toarray().flatten() > 0
myod = data_filtered[:, "MYOD1"].X.toarray().flatten() > 0


In [ ]:
sc.pl.umap(data_filtered[myh7 & myod], color=["MYH7", "MYOD1"])

In [ ]:
sc.pl.pca(muscvar, color=["SYNPO2", "PAX7", "MYOD1", "MYOG", "TRIM63", ], cmap="Greys")

In [ ]:
sc.pl.umap(data, color=["SYNPO2"], cmap="Greys")